<a href="https://colab.research.google.com/github/PriyankaSharma0925/NLP/blob/main/Introduction_to_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow
tensorflow.__version__

'2.8.0'

In [1]:
pip uninstall tensorflow

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.8.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
y
  Successfully uninstalled tensorflow-2.8.0


In [3]:
pip install tensorflow==2.7

     |████████████████████████████████| 489.6 MB 24 kB/s 
     |████████████████████████████████| 463 kB 60.3 MB/s 
     |████████████████████████████████| 1.3 MB 68.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3


In [4]:

# NLP problem also known as seq2seq

#Check the GPU

!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-14f6ef5d-0057-ad21-c08b-49f7d91fd92f)


In [5]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-05-10 06:41:15--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-05-10 06:41:15 (113 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [6]:
from helper_functions import unzip_data, create_tensorboard_callback,plot_loss_curves, compare_historys

In [7]:
#importing the dataset: Kaggle's Introduction to NLP 

!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip


--2022-05-10 06:41:21--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.164.144, 172.253.62.128, 172.253.115.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.164.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2022-05-10 06:41:21 (101 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [8]:
unzip_data('nlp_getting_started.zip')

Visualizing the text data.

In [9]:
import pandas as pd

In [10]:
train_df=pd.read_csv("train.csv")

In [11]:
test_df=pd.read_csv("test.csv")

In [12]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [13]:
shuffled_train_dataset=train_df.sample(frac=1,random_state=42)
shuffled_train_dataset.head()


,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [14]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


**Creating features and target for training dataset.**

In [45]:
X_train=shuffled_train_dataset['text']
X_train
Y_train=shuffled_train_dataset["target"]
len(Y_train)

7613

In [46]:
import tensorflow as tf

In [47]:
from tensorflow.keras.layers import TextVectorization

In [48]:
textvectorizer=TextVectorization(output_sequence_length=15,pad_to_max_tokens=True,max_tokens=1000)

In [49]:
sent=shuffled_train_dataset['text'][0]

In [50]:
textvectorizer.adapt(X_train)

In [51]:
textvectorizer("Its raining heavily outside")

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([ 37,   1,   1, 689,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0])>

In [52]:
import random

In [53]:
rand_sent=random.choice(X_train)
rand_sent

"A little piece I wrote for the Avalanche Designs blog! I'd appreciate it greatly if you checked it out :-)  https://t.co/rfvjh58eF2"

In [54]:
textvectorizer(rand_sent)

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([  3, 259,   1,   8,   1,  10,   2, 676,   1,   1, 493,   1,  15,
         1,  46])>

**Get the total unique words.**

In [55]:
vocab=textvectorizer.get_vocabulary()
print(vocab)

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is', 'for', 'on', 'you', 'my', 'with', 'it', 'that', 'at', 'by', 'this', 'from', 'be', 'are', 'was', 'have', 'like', 'as', 'me', 'up', 'just', 'but', 'so', 'im', 'amp', 'not', 'your', 'out', 'its', 'after', 'will', 'all', 'no', 'fire', 'an', 'has', 'when', 'if', 'we', 'get', 'new', 'now', 'via', 'more', 'about', 'or', 'what', 'people', 'one', 'news', 'he', 'they', 'how', 'dont', 'been', 'over', 'who', 'into', 'do', 'were', 'video', 'us', 'can', '2', 'emergency', 'disaster', 'there', 'police', 'than', 'her', 'would', 'still', 'some', 'his', 'body', 'burning', 'crash', 'back', 'storm', 'california', 'suicide', 'why', 'them', 'off', 'time', 'know', 'got', 'had', 'man', 'buildings', 'day', 'rt', 'first', 'see', 'world', 'going', 'bomb', 'cant', 'nuclear', 'love', 'fires', 'our', 'attack', 'youtube', 'two', '3', 'killed', 'dead', 'go', 'their', 'train', 'full', 'war', 'car', 'being', 'only', 'may', 'families', 'today', 'life', 'hirosh

In [56]:
type(vocab)

list

In [57]:
vocab[-5:]

['date', 'comes', 'camp', 'british', 'bring']

In [58]:
from tensorflow.keras import layers
embedding=layers.Embedding(input_dim=len(vocab),output_dim=128,input_length=15)
embedding

In [59]:
rand_sent=random.choice(X_train)
embedding(textvectorizer(rand_sent))

<tf.Tensor: shape=(15, 128), dtype=float32, numpy=
array([[ 0.03207665, -0.04104109, -0.03388057, ...,  0.00721069,
        -0.03397381,  0.02658899],
       [-0.01279856,  0.01151763,  0.04588201, ..., -0.01174644,
        -0.01923697, -0.01694383],
       [-0.03470466,  0.00537891,  0.01943562, ...,  0.00362482,
         0.04825843,  0.01722636],
       ...,
       [ 0.01530674,  0.00931324,  0.00366434, ..., -0.01205779,
        -0.03370206,  0.02350943],
       [ 0.01530674,  0.00931324,  0.00366434, ..., -0.01205779,
        -0.03370206,  0.02350943],
       [ 0.01530674,  0.00931324,  0.00366434, ..., -0.01205779,
        -0.03370206,  0.02350943]], dtype=float32)>

Model 0 : Getting a baseline. For this we will be using the multinomial naive bayes using the tf-idf

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB 
from sklearn.pipeline import Pipeline

In [61]:
model_0=Pipeline([
                  ("tfidf",TfidfVectorizer()),
                  ("clfier",MultinomialNB())
])
model_0.fit(X_train,Y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clfier', MultinomialNB())])

Evaluate our baseline model.

In [65]:
baseline_score=model_0.score(X_train,Y_train)
print(f"Baseline method has achieved the accuracy of {baseline_score*100:.2f}%")

Baseline method has achieved the accuracy of 88.53%


0.8853277288848023